<a href="https://colab.research.google.com/github/JuanCruzMendoza/AlixPartners-Competencia/blob/main/src/TabTransformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Set Up

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
!pip install pytorch-tabular[all]


In [ ]:

def rolling_sales_stats(df, windows=[7,30,90], col="TOTAL_SALES"):
    """
    Calcula medias y desviaciones móviles para cada SKU-Tienda.
    Considera días sin ventas como 0 dentro de la ventana, excluye el día actual.

    Parámetros:
    - df: DataFrame con columnas ['SKU','STORE_ID','DATE', col]
    - windows: lista de tamaños de ventana
    - col: columna sobre la cual calcular los RF

    Retorna:
    - df con nuevas columnas de medias y desviaciones móviles
    """
    df = df.copy()
    df["DATE"] = pd.to_datetime(df["DATE"])
    df.sort_values(["SKU","STORE_ID","DATE"], inplace=True)

    df["_tmp"] = df[col]

    for w in windows:
        mean_col = f"{col}_mean_{w}D"
        std_col  = f"{col}_std_{w}D"

        def rolling_func(x):
            # Crear rango completo de fechas para incluir días sin ventas
            idx = pd.date_range(start=x.index.min(), end=x.index.max())
            x_full = x.reindex(idx, fill_value=0)
            # Excluir día actual
            rolled = x_full.shift(1).rolling(w, min_periods=1)
            return pd.DataFrame({
                mean_col: rolled.mean(),
                std_col: rolled.std().fillna(0)
            }).reindex(x.index)  # dejar solo filas originales

        rolled_df = (
            df.groupby(["SKU","STORE_ID"])["_tmp"]
              .apply(rolling_func)
              .reset_index(level=[0,1], drop=True)
        )

        df[mean_col] = rolled_df[mean_col].values
        df[std_col]  = rolled_df[std_col].values

    df.drop(columns=["_tmp"], inplace=True)
    return df